In [1]:
print("Fala")

Fala


In [2]:
import pandas as pd

# Análise do CSV _pessoas_completo_

In [3]:
df = pd.read_csv("../data/pessoas_completo.csv")

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030380 entries, 0 to 1030386
Data columns (total 16 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   id_sinistro          1030380 non-null  int64 
 1   municipio            1030380 non-null  object
 2   tipo_via             1030380 non-null  object
 3   tipo_veiculo_vitima  58157 non-null    object
 4   sexo                 1030380 non-null  object
 5   idade                1030380 non-null  Int64 
 6   gravidade_lesao      1030380 non-null  object
 7   tipo_vitima          1030380 non-null  object
 8   faixa_etaria_legal   1030380 non-null  object
 9   profissao            1030380 non-null  object
 10  ano_sinistro         1030380 non-null  int64 
 11  mes_sinistro         1030380 non-null  object
 12  dia_sinistro         1030380 non-null  int64 
 13  ano_obito            58157 non-null    Int64 
 14  mes_obito            58157 non-null    object
 15  dia_obito      

In [5]:
df.describe()

,id_sinistro,idade,ano_sinistro,mes_sinistro,dia_sinistro,ano_obito,mes_obito,dia_obito
count,1.030387e+06,1.013700e+06,1.030387e+06,1.030387e+06,1.030387e+06,58157.000000,58157.000000,58157.000000
mean,1.943842e+06,3.444966e+01,2.021658e+03,6.499242e+00,1.560665e+01,2019.789501,6.497257,15.580119
std,3.947906e+05,1.510009e+01,2.027790e+00,3.376548e+00,8.735699e+00,3.169683,3.378640,8.779323
min,1.256212e+06,0.000000e+00,2.014000e+03,1.000000e+00,1.000000e+00,2015.000000,1.000000,1.000000
25%,1.602038e+06,2.300000e+01,2.020000e+03,4.000000e+00,8.000000e+00,2017.000000,4.000000,8.000000
50%,1.943348e+06,3.100000e+01,2.022000e+03,6.000000e+00,1.600000e+01,2020.000000,6.000000,16.000000
75%,2.278146e+06,4.300000e+01,2.023000e+03,9.000000e+00,2.300000e+01,2023.000000,9.000000,23.000000
max,2.644715e+06,2.700000e+02,2.025000e+03,1.200000e+01,3.100000e+01,2025.000000,12.000000,31.000000


In [53]:
df.head()

,id_sinistro,municipio,tipo_via,tipo_veiculo_vitima,sexo,idade,gravidade_lesao,tipo_vitima,faixa_etaria_legal,profissao,ano_sinistro,mes_sinistro,dia_sinistro,ano_obito,mes_obito,dia_obito
0,2463323,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,51,FATAL,PEDESTRE,50-54,DESCONHECIDA,2015,Janeiro,1,2015,Janeiro,1
1,2465127,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,89,FATAL,PEDESTRE,80 ou mais,DESCONHECIDA,2015,Janeiro,3,2015,Janeiro,3
2,2462123,SAO PAULO,VIAS MUNICIPAIS,PEDESTRE,MASCULINO,67,FATAL,PEDESTRE,65-69,DESCONHECIDA,2015,Janeiro,5,2015,Janeiro,5
3,2506869,MOGI GUACU,VIAS MUNICIPAIS,NAO DISPONIVEL,IGNORADO,44,FATAL,NAO DISPONIVEL,40-44,DESCONHECIDA,2015,Janeiro,9,2015,Janeiro,9
4,2490194,COTIA,RODOVIAS,AUTOMOVEL,MASCULINO,28,FATAL,CONDUTOR,25-29,DESCONHECIDA,2015,Janeiro,11,2015,Janeiro,11


In [7]:
df.count()

id_sinistro                 1030387
municipio                   1030380
tipo_via                    1030387
tipo_veiculo_vitima           58157
sexo                        1030387
idade                       1013700
data_obito                    58157
gravidade_lesao             1030387
tipo_de vítima               992638
faixa_etaria_demografica    1030387
faixa_etaria_legal          1030387
profissao                    852894
data_sinistro               1030387
ano_sinistro                1030387
mes_sinistro                1030387
dia_sinistro                1030387
ano_mes_sinistro            1030387
ano_obito                     58157
mes_obito                     58157
dia_obito                     58157
ano_mes_obito                 58157
dtype: int64

In [30]:
df.isna().sum()

id_sinistro                      0
municipio                        0
tipo_via                         0
tipo_veiculo_vitima         972223
sexo                             0
idade                            0
gravidade_lesao                  0
tipo_vitima                      0
faixa_etaria_demografica         0
faixa_etaria_legal               0
profissao                        0
ano_sinistro                     0
mes_sinistro                     0
dia_sinistro                     0
ano_obito                   972223
mes_obito                   972223
dia_obito                   972223
dtype: int64

In [31]:
df["mes_obito"].isna().sum()

972223

In [50]:
df["faixa_etaria_legal"].value_counts()

18-24             255250
25-29             151949
30-34             119825
35-39             105705
40-44              89427
45-49              67832
0-17               57686
50-54              52030
55-59              38858
60-64              27391
65-69              18895
NAO DISPONIVEL     16687
70-74              12662
75-79               8355
80 ou mais          7828
Name: faixa_etaria_legal, dtype: int64

## Removendo colunas e linhas
Alguns dados da tabela estão com valores desnecessários pra análise

Verificando se todos os acidentes que são fatais contém data do óbito

In [11]:
((df["gravidade_lesao"] == "FATAL") & (df["data_obito"].isna())).sum()

0

Removendo colunas **ano_mes_obito, ano_mes_sinistro, data_obito** e **data_sinistro** pois são redundantes

In [12]:
df = df.drop(columns=["ano_mes_sinistro", "ano_mes_obito", "data_obito", "data_sinistro"])

Removendo linhas com valor nulo nas colunas **municipio** e **tipo_via** 

In [13]:
df = df.dropna(subset=["municipio", "tipo_via"])

Verificando se **faixa_etaria_demografica** e **faixa_etaria_legal** são redundantes

In [48]:
print("faixa demográfica:", df["faixa_etaria_demografica"].unique())
print("faixa legal:", df["faixa_etaria_legal"].unique())

faixa demográfica: ['50 a 54' '85 a 89' '65 a 69' '40 a 44' '25 a 29' '60 a 64' '05 a 09'
 '30 a 34' '20 a 24' '15 a 19' '00 a 04' '70 a 74' '75 a 79'
 'NAO DISPONIVEL' '35 a 39' '55 a 59' '45 a 49' '80 a 84' '10 a 14'
 '90 e +']
faixa legal: ['50-54' '80 ou mais' '65-69' '40-44' '25-29' '60-64' '0-17' '30-34'
 '18-24' '70-74' '75-79' 'NAO DISPONIVEL' '35-39' '55-59' '45-49']


Como os valores de ambas colunas são fortemente semelhantes optarei por manter apenas uma delas

In [51]:
df = df.drop("faixa_etaria_demografica", axis=1)

## Renomeação e conversão de colunas e valores de linhas
Muitos dados precisam ser tratados para poderem ser analisados mais apropriadamente

Renomeando coluna **tipo_de vítima** para **tipo_vitima**

In [14]:
df = df.rename(columns={"tipo_de vítima": "tipo_vitima"})

Adicionando idade média nos valores nulos da coluna **idade** e renomeando valores nulos das colunas **profissao** e **tipo_vitima**

In [15]:
df["idade"] = df["idade"].fillna(df["idade"].mean())
df["profissao"] = df["profissao"].fillna("DESCONHECIDA")
df["tipo_vitima"] = df["tipo_vitima"].fillna("NAO DISPONIVEL")

Renomeando valores _nao disponivel_ da coluna **sexo** para _ignorado_

In [16]:
df["sexo"] = df["sexo"].replace({"NAO DISPONIVEL": "IGNORADO"})

Mudando tipo de dado das colunas **idade, mes_obito, mes_sinistro, ano_obito** e **dia_obito** de `float64` para `int64`

In [26]:
df["idade"] = df["idade"].round().astype("Int64")
df["mes_sinistro"] = df["mes_sinistro"].astype("Int64")
df["mes_obito"] = df["mes_obito"].astype("Int64")
df["ano_obito"] = df["ano_obito"].astype("Int64")
df["dia_obito"] = df["dia_obito"].astype("Int64")

mudando valores mensais das colunas **mes_sinistro** e **mes_obito** de numérico para texto

In [29]:
month_convert = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril", 5: "Maio", 6: "Junho",
    7: "Julho", 8: "Agosto", 9: "Setembro", 10: "Outubro", 11: "Novemrbo", 12: "Dezembro"
}
df["mes_obito"] = df["mes_obito"].astype("Int64").map(month_convert)
df["mes_sinistro"] = df["mes_sinistro"].astype("Int64").map(month_convert)

# Analise do CSV _sinistros_completo_ 

In [3]:
df = pd.read_csv("../data/sinistros_completo.csv")

In [9]:
df.count()

id_sinistro                        1240943
tipo_registro                      1240943
data_sinistro                      1240943
ano_sinistro                       1240943
mes_sinistro                       1240943
dia_sinistro                       1240943
ano_mes_sinistro                   1240943
hora_sinistro                      1237540
logradouro                         1237740
numero_logradouro                  1231393
tipo_via                           1240943
latitude                           1234042
longitude                          1234043
municipio                          1240936
regiao_administrativa              1240936
tp_veiculo_bicicleta                 30757
tp_veiculo_caminhao                  50243
tp_veiculo_motocicleta              483506
tp_veiculo_nao_disponivel           143726
tp_veiculo_onibus                    30573
tp_veiculo_outros                    27729
tp_veiculo_automovel                512769
gravidade_nao_disponivel            109161
gravidade_l

In [16]:
df["tp_sinistro_colisao_frontal"].value_counts()

S    154185
Name: tp_sinistro_colisao_frontal, dtype: int64

Criar CSV com o dataset

In [18]:
#df.to_csv("pessoas_completo_limpo.csv", index=False)